In [1]:
import sys
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
from sklearn.svm import SVR
from score_metrics import rmse, spearman, mcc, multiclass_mcc

from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
def get_features_df(activities_file, compound_ft_file, target_ft_file=None):
    activities = pd.read_csv(activities_file, sep=',')
    compounds = pd.read_csv(compound_ft_file, sep=',')
    activities = activities[activities["type"] == "IC50"]
    activities = activities[["target_chembl_id",
                             "molecule_chembl_id", "standard_value"]].dropna()
    features = activities.merge(compounds, on="molecule_chembl_id")
    features["standard_value"] = features["standard_value"]/1000

    if target_ft_file:
        targets = pd.read_csv(target_ft_file, sep=',', header=None)
        targets.columns = ["target_chembl_id"] + \
            [f"prot_ft_{i}" for i in range(targets.shape[1]-1)]
        features = features[features["target_chembl_id"].isin(
            targets["target_chembl_id"])]
        features = features.merge(targets, on="target_chembl_id")

    features.to_csv("features.csv", index=False)
    return features

In [10]:
def SVR_model(input):
    input = input.sample(frac=1).reset_index(drop=True)
    y = input["standard_value"]
    X_train, X_test, y_train, y_test = train_test_split(input, y, test_size=0.2, random_state=0)

    if "prot_ft_0" in X_train.columns: # if we use protein features
        prot_cols = [col for col in X_train.columns if col.startswith("prot_ft")]
        scaler = MinMaxScaler()
        X_train[prot_cols] = scaler.fit_transform(X_train[prot_cols])
        X_test[prot_cols] = scaler.transform(X_test[prot_cols])

    # find best parameters
    param_grid = {'C': [0.1, 10, 1000, 5000, 10000, 50000, 100000], 'gamma': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1]}
    gs = GridSearchCV(SVR(), param_grid, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')
    gs.fit(X_train.iloc[:, 3:], y_train)
    print("Grid Search")
    print("best params: ", gs.best_params_)
    print(pd.concat([pd.DataFrame(gs.cv_results_["params"]), pd.DataFrame(-gs.cv_results_["mean_test_score"], columns=["RMSE"])],axis=1))

    # test
    model = SVR(C=gs.best_params_['C'], gamma=gs.best_params_['gamma'])
    model.fit(X_train.iloc[:, 3:], y_train)
    pred = model.predict(X_test.iloc[:, 3:])
    test_result = X_test.iloc[:, :3]
    test_result['predicted'] = pred
    test_result.to_csv("SVR_model_test.tsv", sep="\t", index=False)
    print("Test set results:")
    print(f"RMSE: {rmse(pred, y_test)}, Spearman: {spearman(pred, y_test)}, MCC: {mcc(pred, y_test.to_numpy(), np.median(y_test))}")
    return model


In [7]:
def RF_model(input):
    input = input.sample(frac=1).reset_index(drop=True)
    y = input["standard_value"]
    X_train, X_test, y_train, y_test = train_test_split(input, y, test_size=0.2, random_state=0)

    if "prot_ft_0" in X_train.columns: # if we use protein features
        prot_cols = [col for col in X_train.columns if col.startswith("prot_ft")]
        scaler = MinMaxScaler()
        X_train[prot_cols] = scaler.fit_transform(X_train[prot_cols])
        X_test[prot_cols] = scaler.transform(X_test[prot_cols])

    # find best parameters
    param_grid = {'n_estimators': [10, 100, 1000], 'max_features': ['sqrt', 'log2'], 'max_depth': [5, 10, 20, 50]}
    gs = GridSearchCV(RandomForestRegressor(), param_grid, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error', verbose=4)
    gs.fit(X_train.iloc[:, 3:], y_train)
    print("Grid Search")
    print("best params: ", gs.best_params_)
    print(pd.concat([pd.DataFrame(gs.cv_results_["params"]), pd.DataFrame(-gs.cv_results_["mean_test_score"], columns=["RMSE"])],axis=1))

    # test
    model = RandomForestRegressor(n_estimators=gs.best_params_['n_estimators'], max_features=gs.best_params_['max_features'], max_depth=gs.best_params_['max_depth'])
    model.fit(X_train.iloc[:, 3:], y_train)
    pred = model.predict(X_test.iloc[:, 3:])
    test_result = X_test.iloc[:, :3]
    test_result['predicted'] = pred
    test_result.to_csv("RF_model_test.tsv", sep="\t", index=False)
    print("Test set results:")
    print(f"RMSE: {rmse(pred, y_test)}, Spearman: {spearman(pred, y_test)}, MCC: {mcc(pred, y_test.to_numpy(), np.median(y_test))}")
    return model


In [11]:
activities_file = "activities.csv"
compound_ft_file = "ligands_ecfp4.csv"
target_ft_file = "targets_embedding.csv"

input = get_features_df(activities_file, compound_ft_file, target_ft_file)
svr = SVR_model(input)


Grid Search
best params:  {'C': 50000, 'gamma': 0.05}
           C  gamma       RMSE
0        0.1  0.001  28.552355
1        0.1  0.005  28.104035
2        0.1  0.010  28.157924
3        0.1  0.050  28.602292
4        0.1  0.100  28.702807
5        0.1  0.500  28.769331
6        0.1  1.000  28.769687
7       10.0  0.001  25.653438
8       10.0  0.005  24.047716
9       10.0  0.010  23.173627
10      10.0  0.050  21.555299
11      10.0  0.100  22.061921
12      10.0  0.500  24.012561
13      10.0  1.000  24.032781
14    1000.0  0.001  17.774351
15    1000.0  0.005  17.831181
16    1000.0  0.010  17.601476
17    1000.0  0.050  17.458692
18    1000.0  0.100  17.972520
19    1000.0  0.500  18.767075
20    1000.0  1.000  18.780880
21    5000.0  0.001  17.796830
22    5000.0  0.005  17.783363
23    5000.0  0.010  17.601476
24    5000.0  0.050  17.458692
25    5000.0  0.100  17.972520
26    5000.0  0.500  18.767075
27    5000.0  1.000  18.780879
28   10000.0  0.001  18.296952
29   10000.0  0.

In [9]:
rf = RF_model(input)


Fitting 10 folds for each of 24 candidates, totalling 240 fits
Grid Search
best params:  {'max_depth': 50, 'max_features': 'sqrt', 'n_estimators': 10}
    max_depth max_features  n_estimators       RMSE
0           5         sqrt            10  22.622332
1           5         sqrt           100  21.923000
2           5         sqrt          1000  21.864957
3           5         log2            10  22.579709
4           5         log2           100  21.905901
5           5         log2          1000  22.003245
6          10         sqrt            10  19.949947
7          10         sqrt           100  20.081952
8          10         sqrt          1000  20.386452
9          10         log2            10  20.643180
10         10         log2           100  20.416043
11         10         log2          1000  20.418404
12         20         sqrt            10  20.649639
13         20         sqrt           100  19.858678
14         20         sqrt          1000  19.891766
15         20    